In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from robotpal import Robot, Camera, bgr8_to_jpeg

from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time
from robotpal.SCSCtrl import TTLServo

robot = Robot()

TTLServo.servoAngleCtrl(5, -25, 1, 100)
print('Camera Ready!')

DATASET_DIR = 'dataset_xy_test4'

try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

[RobotPal] 서버 시작됨 | WebSocket: 9999, TCP: 9998
[Send Servo] ID: 5, Angle: 24.988235294117647, Speed: 20.0
Camera Ready!
Directories not created becasue they already exist
[RobotPal] 통신 대기 중... (WS: 9999, TCP: 9998)


[System] WebSocket 프로세서 시작
[TCP] 클라이언트 연결됨 (('127.0.0.1', 51384))


In [2]:
camera = Camera()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

def display_xy(camera_image):
    # 1. 이미지를 224x224 정사이즈로 리사이즈 (데이터 수집용 표준 크기)
    image = cv2.resize(camera_image, (224, 224))
    
    # 2. 224 해상도에서 '굵고 뚜렷하게' 보이는 값으로 설정 (원상복구)
    # 800일 때 10/25였던 비율을 224에 맞추면 3/8이 적당합니다.
    thickness = 3
    radius = 8

    x = x_slider.value
    y = y_slider.value
    
    # 3. 좌표 계산 (224 크기 기준)
    # 중앙값 112를 기준으로 계산
    x = int(x * 112 + 112)
    y = int(y * 112 + 112)
    
    # 차량 기준점 (화면 하단 중앙)
    # 224가 끝이므로 220 정도로 설정해 점이 짤리지 않게 함
    car_x = 112
    car_y = 224 
    
    # 그리기
    image = cv2.circle(image, (x, y), radius, (0, 255, 0), thickness)          # 녹색 점 (목표)
    image = cv2.circle(image, (car_x, car_y), radius, (0, 0, 255), thickness)  # 빨간 점 (내 위치)
    image = cv2.line(image, (x, y), (car_x, car_y), (255, 0, 0), thickness)    # 파란 선
    
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

# --- 이하 로직은 동일 ---
camera = Camera()
time.sleep(1)

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

display(widgets.HBox([image_widget, target_widget]))

count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))
save_button = widgets.Button(description='Save', button_style='success')

display(widgets.VBox([x_slider, y_slider, count_widget, save_button]))

def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

def save_snapshot():
    uuid = xy_uuid(x_slider.value, y_slider.value)
    image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_widget.value) # 224x224 이미지가 저장됩니다.
    count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))
    
save_button.on_click(lambda x: save_snapshot())

# 버튼 컨트롤 생성
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])
display(controls_box)

def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(0.4)

def step_backward(change):
    robot.backward(0.4)

def step_left(change):
    robot.left(0.3)
    time.sleep(0.5)
    robot.stop()

def step_right(change):
    robot.right(0.3)
    time.sleep(0.5)
    robot.stop()

stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

In [8]:
camera.unobserve_all()
camera.stop()
robot.stop()

## 데이터 압축해서 내보내기
data 파일을 외부 클라우드 머신에 옮기고 싶다면, 아래 명령어를 이용해서 압축한다.  
이제 다음 train_model 로 이동합니다.

In [ ]:
!zip -r -q road_following_{DATASET_DIR}.zip {DATASET_DIR}